# 11장 감정분석(Sentiment Analysis)




#11-1 사전기반 감정분석

In [1]:
#KnuSentiLex 다운로드
!wget https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/pos_pol_word.txt
!wget https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/neg_pol_word.txt

--2020-07-27 00:48:17--  https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/pos_pol_word.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85007 (83K) [text/plain]
Saving to: ‘pos_pol_word.txt’

pos_pol_word.txt    100%[===================>]  83.01K  --.-KB/s    in 0.03s   

2020-07-27 00:48:17 (2.66 MB/s) - ‘pos_pol_word.txt’ saved [85007/85007]

--2020-07-27 00:48:18--  https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/neg_pol_word.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176542 (172K) [text/plain

In [4]:
# !cat neg_pol_word.txt # 이건 그냥 보는거
!head -50 neg_pol_word.txt

출처
1) 표준국어대사전 뜻풀이
-> https://github.com/mrchypark/stdkor
2) 신조어
-> https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%8B%A0%EC%A1%B0%EC%96%B4_%EB%AA%A9%EB%A1%9D
3) 이모티콘
-> https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%AA%A8%ED%8B%B0%EC%BD%98
4) SentiWordNet_3.0.0_20130122
-> http://sentiwordnet.isti.cnr.it/
5) SenticNet-5.0
-> http://sentic.net/
6) 감정단어사전0603
-> http://datascience.khu.ac.kr/board/bbs/board.php?bo_table=05_01&wr_id=91 
7) 김은영, “국어 감정동사 연구”, 2004.02, 학위논문(박사) - 전남대학교 국어국문학과 대학원


단어 수 : 9826
극성 : 부정

가난
가난뱅이
가난살이
가난살이하다
가난설음
가난에
가난에 쪼들려서
가난하게
가난하고
가난하고 어렵다
가난하고 천한
가난하다
가난하여
가난한
가난한 데다가
가난한 사람을
가난히
가년스럽다
가녕스럽다
가누지 못하고
가누지 못하다
가누지 못할
가누지 못할 정도로
가누지 못함
가눌 수 없게
가눌 수 없을
가눌 수 없이
가당찮다
가당찮이
가두거나 해치거나
가망이 없게


In [9]:
dct = {}

import pandas as pd
# 긍정단어 조회

with open('pos_pol_word.txt', 'r') as f:
  dct['pos'] = f.read().split('\n')[19:]

# 부정단어 조회

with open('neg_pol_word.txt', 'r') as f:
  dct['neg'] = f.read().split('\n')[19:]

print(dct['pos'][:10])
print(dct['neg'][:10])



['(-;', '(^^)', '(^-^)', '(^^*', '(^_^)', '(^o^)', '*^^*', '/^o^\\', ':(', ":'-("]
['가난', '가난뱅이', '가난살이', '가난살이하다', '가난설음', '가난에', '가난에 쪼들려서', '가난하게', '가난하고', '가난하고 어렵다']


In [11]:
txt = "코로나19의 여파로 전 세계 교사들과 학생들 모두 혼란스러운 시기를 보내고 있습니다. 초유의 상황에서도, 학생들을 위해 최선을 다하려는 선생님들의 노력이 계속되고 있는데요. 어려움 속에서, 선생님들은 어떤 방법으로 사랑을 전하고 있을까요? 뉴스G에서 전해드립니다. [리포트] 중국 저장성의 한 산골 마을. 초등학교 교사 왕진량 씨는 지난 2월 말부터, 매일 새벽 다섯 시면 부지런히 집을 나섭니다. 온라인 수업을 받을 수 없는, 깊은 산골 마을에 살고 있는 학생들을 찾아가기 위해서인데요. 선생님이 하루 동안 이동하는 거리는 대략 30km정도. 차도 없이 도보로 네 개의 마을을 돌아다니며 학생들을 만납니다. 매일 이어지는 강행군이지만, 오로지 아이들의 학습이 중단되어서는 안 된다는 생각뿐입니다. 혹시 모를 사태에 대비해 학생들과의 접촉은 최대한 줄입니다. 숙제를 내주고, 검사 후에 모르는 문제를 알려주는 식으로 일대일 수업을 진행하고 있는데요. 아이들에게 배우는 즐거움이 얼마나 소중한 것인지 잘 알기에, 선생님은 이렇게라도 수업을 할 수 있다는 데서 행복을 느낍니다. 영국의 한 초등학교 교사인 젠 포울스 씨는 매일 아침, 무거운 짐을 앞 뒤, 양 옆으로 짊어지고 씩씩하게 발걸음을 옮깁니다. 코로나19로 학교가 문을 닫은 뒤, 형편이 어려운 학생들을 위해 매일 78인분의 점심 도시락을 배달하고 있는데요. 선생님이 재직 중인 초등학교는 전체 학생의 41퍼센트가 무상 급식 대상자이기 때문입니다. 도시락의 무게는 18kg, 걸어야 하는 거리는 8km에 달하지만 기다리는 학생들을 생각하며 지치지 않고 발걸음을 재촉합니다. 학생들은 창문을 통해서 반갑게 인사하기도 하고, 선생님이 볼 수 있게 감사 메시지를 붙여 놓기도 하는데요. 선생님이 정성껏 준비해 손수 배달한 사랑의 도시락. 봉쇄된 도시의 굶주린 아이들에게 소중한 한 끼 식사, 그 이상의 의미가 되고 있습니다. 미국 사우스다코타 주의 중학교 수학 교사인 크리스 와바 씨는, 커다란 화이트보드를 들고 학생의 집을 찾았습니다. 온라인 수업 후, 학생에게 이메일로 방정식 풀이법에 대한 질문을 받았기 때문인데요. 이메일로 답변해주는 것보다 직접 풀이 과정을 보여주는 게 낫다는 생각이었죠. 깜짝 놀란 학생을 마주한 채, 선생님은 현관문 앞에서 열정적으로 문제를 풀기 시작했습니다. 이 열정적인 강의는, 학생이 풀이법을 완벽히 이해할 때까지 이어졌는데요. 바이러스는 전 세계 교실에 혼란을 불러왔지만, 선생님들의 노력은 저마다의 방식으로 계속되고 있습니다. 어려움 속에서도 학생들을 위해 안간힘을 쓰고 있는 모든 선생님들에게, 응원과 박수를 함께 보냅니다."

In [10]:
def analye_sentiment(txt, dct) :
  pos = []
  neg = []
  for p in dct['pos']:
    if p in txt:
      pos.append(p)

  for n in dct['neg']:
    if n in txt:
      neg.append(n)

  return (len(pos)/(len(pos)+len(neg))), (len(neg)/(len(pos) + len(neg)))

In [12]:
pos, neg = analye_sentiment(txt, dct)
print(pos)
print(neg)

0.6470588235294118
0.35294117647058826


# 11-2 나이브 베이즈 분류기 활용 감정분석

## 2.1 간단 예제

In [13]:
text = [("I love you", "P"),
        ("love happy weekend", "P"),
        ("bore work job", "N"),
        ("I hate you", "N"),
        ("bore weekend", "N"),
        ("happy together", "P")]

In [17]:
X_train = [t[0] for t in text]
y_train = [t[1] for t in text]
X_train

['I love you',
 'love happy weekend',
 'bore work job',
 'I hate you',
 'bore weekend',
 'happy together']

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB



In [18]:
text_clf = Pipeline([
                ('vect', CountVectorizer()),
                ('tfidf',TfidfTransformer()), 
                ('clf', MultinomialNB())
                ])
text_clf.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
## 성생님
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
clf = MultinomialNB().fit(X_train_counts, y_train)

In [47]:
count_vect = CountVectorizer()
count_vect.fit_transform(X_train, y_train)

<6x9 sparse matrix of type '<class 'numpy.int64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [48]:
import numpy as np
count_vect.transform(X_train).toarray()

array([[0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0]])

In [ ]:
count_vect.transform

In [49]:
count_vect.transform(["happy weekend"]).toarray()

array([[0, 1, 0, 0, 0, 0, 1, 0, 0]])

In [58]:
count_vect.transform(X_train).toarray().shape

(6, 9)

In [60]:
clf = MultinomialNB()
clf.fit(count_vect.transform(X_train).toarray(), np.array(y_train).reshape(-1,1))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [63]:
clf = MultinomialNB()
clf.fit(count_vect.transform(X_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [64]:
print(clf.predict(count_vect.transform(["happy weekend"])))
print(clf.predict_proba(count_vect.transform(["happy weekend"])))

['P']
[[0.25 0.75]]


## 2.2 네이버 영화 리뷰 감정분석

In [62]:
# 네이버 영화 리뷰 다운로드
!wget https://github.com/e9t/nsmc/raw/master/ratings.txt

--2020-07-27 01:38:26--  https://github.com/e9t/nsmc/raw/master/ratings.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt [following]
--2020-07-27 01:38:27--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘ratings.txt’

ratings.txt         100%[===================>]  18.61M  27.4MB/s    in 0.7s    

2020-07-27 01:38:28 (27.4 MB/s) - ‘ratings.txt’ saved [19515078/19515078]



In [65]:
!head -5 ratings.txt

id	document	label
8112052	어릴때보고 지금다시봐도 재밌어요ㅋㅋ	1
8132799	디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.	1
4655635	폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.	1
9251303	와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지	1


In [102]:
# 사실, txt file도 binary file이 아니기 때문에,(Csv도 그냥 텍스트) -> read_csv가능
import pandas as pd
import numpy as np

df = pd.read_csv('ratings.txt', delimiter='\t', index_col= None) 


In [103]:
df.columns

Index(['id', 'document', 'label'], dtype='object')

In [118]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df['document'], df['label'], random_state =0)


In [114]:
X_train[:10]

86283     킬링타임용으로 딱이다. 뻔한 전개이긴 해도, 원래 로코가 다 그렇기 때문에. 보면서...
70198                                   정말 기억에남는 유쾌한 드라마 ㅎㅎ
152765                       시간낭비했다. 갈수록 제작비 아낄려고 용쓰는게 보인다.
21563                    권총 속 장전된 탄환처럼 발사되어 연기만 남긴 채 사라진 청춘
123869                1부에 비해 너무 떨어진다...촬영에 문제는 없지만,몰입도가 현저히
33860     배우들의 감탄할 수밖에 없는 연기와 음악 영상 모든게 너무 좋았고 매력적이었어요.....
75191                                        추억영화 ㅜ ㅜ 짱재밌어요
4908          내용전개가 억지스러운게 좀 있지만 소소하게 웃을만하고 김옥빈 춤추는거 진짜 이뻤다
110481                                     Tmfprresrserfser
14638     오늘 막방 너무 슬펐어요. 꽃들의 전쟁 모든 연기자분들 수고 많았어요. 다음 작품 ...
Name: document, dtype: object

In [121]:
df.describe()

,id,label
count,2.000000e+05,200000.000000
mean,6.739774e+06,0.500000
std,2.923504e+06,0.500001
min,3.300000e+01,0.000000
25%,4.760678e+06,0.000000
50%,7.527726e+06,0.500000
75%,9.248413e+06,1.000000
max,1.027815e+07,1.000000


In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [123]:
df = pd.read_csv('./ratings.txt', sep='\t').dropna() # 이게 제일 중요
df.tail(10)

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df['document'], df['label'], random_state=0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [122]:
X_train, X_test, y_train, y_test = train_test_split(df['document'], df['label'], test_size = 0.25, random_state =0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

ValueError: ignored

In [126]:
# X_new_counts = count_vect.transform(X_test)
# X_new_tfidf = count_vect.transform(X_new_counts)
# predict = clf.predict(X_new_tfidf)
# np.mean(predicted == y_test)

X_new_counts = count_vect.transform(X_test)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)
np.mean(predicted == y_test)

0.8322332893315733

In [127]:
predicted[:10]

array([1, 1, 0, 1, 1, 1, 0, 1, 1, 0])

In [128]:
X_test[:10]

78888                     진짜 개웃김 ㅋㅋㅋ 이런 코드너무 좋다 언능 2나왔음 좋겠다
49058     고속버스타고 가는 길에 핸드폰으로 보다가 울었네요 ㅠㅠ 아 슬픈듯.. 개인적으로 진...
101711                           개성 없는 모방 일색의 유치하고 가벼운 오락물.
97260     상처가없는이들에겐 갖혀있는소녀의 저질액션 미친영화이겠으나 나에겐 꿈처럼시작해서 미련...
53590                               아 다시 보고 싶은데 디비디가 안나온다..
62095                                            김민종씨 멋집니다.
36349          무식하게 거대한 원양어선으로 대량포획하는 미국, 일본 등등 국가들은 반성해라!!
151288                                        콜럼버스는 사기꾼이었다!
88072                                           재미있어요 재미있어요
117532    일주일에 한번봐도 대충 내용이 감이 오는 뻔한 스토리젤 짜증나는건 민세현? 연기 못...
Name: document, dtype: object

In [112]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([
                ('vect', CountVectorizer()),
                ('tfidf',TfidfTransformer()), 
                ('clf', MultinomialNB())
                ])

text_clf.fit(X_train.astype('U').values, y_train.astype('U').values)

KeyboardInterrupt: ignored

In [97]:
y_train.head(10)

105800     8681130
27538       959405
108979     6201284
184818    10129710
121763     8498124
33713      6025228
31780       709617
76733     10124730
198671     8663977
17365      9850907
Name: id, dtype: int64